In [ ]:
# Célula 1: Importações de Bibliotecas e Módulos

import pandas as pd
from sklearn import impute, model_selection
from sklearn.experimental import enable_iterative_imputer # Necessário para o IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyClassifier
import missingno as msno

In [ ]:
# Célula 2: Carga de Dados e Inspeção
df = pd.read_excel('titanic3.xls')

# Inspeção inicial (opcional, mas bom para checagem)
print("Formato dos dados:", df.shape)
print("Cabeçalho:")
df.head()

In [ ]:
# Célula 3: Definições das Funções do Pipeline (COM CORREÇÃO DO WARNING)

def tweak_titanic(df):
    """Realiza a limpeza inicial e o one-hot encoding."""
    cols_to_drop = [
        'name', 'ticket', 'home.dest', 'boat', 'body', 'cabin',
    ]
    df = df.drop(columns=cols_to_drop)
    
    # Preenche NaNs na coluna 'embarked' (Categórica) com a Moda antes de fazer o get_dummies
    df['embarked'] = df['embarked'].fillna(df['embarked'].mode()[0])
    
    df = pd.get_dummies(df, drop_first=True)
    return df

def get_train_test_X_y(df, y_col, size=0.3, std_cols=None):
    """Divide, Imputa (Iterative) e Escala os dados de treino e teste."""
    
    y = df[y_col]
    X = df.drop(columns=y_col)
    X_train, X_test, y_train, y_test = model_selection.train_test_split(
        X, y, test_size=size, random_state=42
    )
    
    # Colunas que serão imputadas com IterativeImputer (apenas as de fato numéricas)
    num_cols_to_impute = ['age', 'fare'] 
    
    # 1. Imputação de Valores Ausentes
    fi = impute.IterativeImputer(random_state=42)
    X_train.loc[:, num_cols_to_impute] = fi.fit_transform(X_train[num_cols_to_impute])
    X_test.loc[:, num_cols_to_impute] = fi.transform(X_test[num_cols_to_impute])
    
    # 2. Preenchimento de qualquer NaN remanescente com a mediana
    meds = X_train.median()
    X_train = X_train.fillna(meds)
    X_test = X_test.fillna(meds)
    
    # 3. Escalonamento (BLOCO CORRIGIDO AQUI)
    if std_cols:
        std = StandardScaler()
        # Converte para float antes da atribuição para evitar o FutureWarning
        X_train.loc[:, std_cols] = std.fit_transform(X_train[std_cols].astype(float))
        X_test.loc[:, std_cols] = std.transform(X_test[std_cols].astype(float))
        
    return X_train, X_test, y_train, y_test

In [ ]:
# Célula 4: Aplicação do Pipeline e Divisão Final

# 1. Aplica a limpeza e o encoding
ti_df = tweak_titanic(df) 

# 2. Colunas para Escalonamento
std_cols = "pclass age sibsp fare".split() 

# 3. Executa a divisão, imputação e escalonamento
X_train, X_test, y_train, y_test = get_train_test_X_y(
    ti_df, "survived", std_cols=std_cols
)

# Checagem final (opcional)
print("\nFormato do conjunto de Treino:", X_train.shape)
print("Valores ausentes em X_train:", X_train.isnull().sum().sum())

In [ ]:
# Célula 5: Modelagem e Avaliação (Modelo Baseline)

# Cria e treina o modelo Dummy
bm = DummyClassifier() 
bm.fit(X_train, y_train)

# Avalia a acurácia
accuracy = bm.score(X_test, y_test) 

print(f"Acurácia do modelo Dummy (Baseline): {accuracy:.4f}")

In [ ]:
# Célula 5: Comparação de Modelos com Validação Cruzada (CORRIGIDA)

# 1. Recombina os dados (necessário para a Validação Cruzada)
X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])

# 2. Importações de modelos (Garantindo que estão todos aqui)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
import xgboost
import inspect # Importa o módulo inspect para verificar os argumentos aceitos

# 3. Lista de Modelos
MODELS = [
    DummyClassifier, 
    LogisticRegression,
    DecisionTreeClassifier,
    KNeighborsClassifier,
    GaussianNB,
    SVC,
    RandomForestClassifier,
    xgboost.XGBClassifier,
]

# 4. Loop de Comparação de Modelos
kfold = model_selection.KFold(n_splits=10, random_state=42, shuffle=True)
print(f"{'Modelo':22} {'AUC':5} {'STD':5}")
print("-" * 32)


for model in MODELS:
    # 5. Lógica de Inicialização: Verifica se o modelo aceita 'random_state'
    
    # Obtém os parâmetros que o construtor do modelo (o __init__) aceita
    params = inspect.signature(model.__init__).parameters
    
    if 'random_state' in params:
        # Inicializa o modelo passando random_state=42
        cls = model(random_state=42)
    else:
        # Inicializa o modelo sem o argumento random_state
        cls = model() 
    
    # 6. Validação Cruzada
    s = model_selection.cross_val_score(cls, X, y, scoring="roc_auc", cv=kfold)
    
    # 7. Imprime o resultado
    print(f"{model.__name__:22} AUC: "
          f"{s.mean():.3f} STD: {s.std():.2f}")

In [ ]:
# Célula 7: Treinamento e Avaliação do Random Forest (Com n_estimators=10)

from sklearn.ensemble import RandomForestClassifier # Importação limpa e direta

# 1. Cria o objeto do modelo
rf = RandomForestClassifier(
    n_estimators=10,  # Usando 10 estimadores
    random_state=42
)

# 2. Treinamento do modelo
rf.fit(X_train, y_train) 

# 3. Avaliação no conjunto de teste
score = rf.score(X_test, y_test)

# 4. Imprime o resultado
print(f"Acurácia do RandomForest (no conjunto de teste, com 10 árvores): {score:.4f}")

In [ ]:
# Célula 8: Grid Search (Ajuste de Hiperparâmetros - TENTANDO REDUZIR OVERFITTING)

from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
# X_train e y_train devem estar carregados na memória

# 1. Inicializa o modelo base (COM random_state fixo)
rf4 = RandomForestClassifier(random_state=42)

# 2. Define o dicionário de parâmetros a serem testados
# NOVOS VALORES: Inclusão de max_depth e min_samples_leaf mais restritivos
params = {
    "max_features": [0.4, "sqrt"], 
    "n_estimators": [15, 200],
    
    # Valores mais restritivos para forçar a generalização (reduzir Overfitting)
    "min_samples_leaf": [5, 10, 20], 
    
    # NOVO PARÂMETRO: Limita a profundidade das árvores (muito eficaz contra Overfitting)
    "max_depth": [5, 8, None] # 5 e 8 são limites, None deixa ir até o fim
} 

# 3. Executa o Grid Search
cv = model_selection.GridSearchCV(
    rf4, 
    params, 
    n_jobs=-1, 
    scoring="accuracy", # Usando 'accuracy' como métrica, como no original
).fit(X_train, y_train)

# 4. Imprime os melhores parâmetros encontrados
print("Melhores parâmetros encontrados (tentando reduzir overfitting):")
print(cv.best_params_)

# 5. Armazena o melhor modelo encontrado
rf5 = cv.best_estimator_

In [ ]:
# Célula 9: Treinamento e Avaliação do RF Otimizado

# 1. Cria o objeto do modelo usando os melhores parâmetros do Grid Search
# O operador ** desempacota o dicionário cv.best_params_ em argumentos nomeados
# Isso garante que você use a combinação {'max_features': 'auto', 'min_samples_leaf': 0.1, 'n_estimators': 200}
rf5 = RandomForestClassifier(
    **cv.best_params_, 
    random_state=42 # Adiciona o random_state para reprodutibilidade
)

# 2. Treinamento
rf5.fit(X_train, y_train)

# 3. Avaliação no conjunto de teste
score = rf5.score(X_test, y_test)

# 4. Imprime o resultado
print(f"Acurácia do Random Forest Otimizado: {score:.4f}")

In [ ]:
# Alternativa: Usando apenas Scikit-learn (Não depende de distutils)
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

y_pred = rf5.predict(X_test)
cm = confusion_matrix(y_test, y_pred, labels=np.unique(y_test))

disp = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=["Morreu (0)", "Sobreviveu (1)"]
)

fig, ax = plt.subplots(figsize=(8, 8))
disp.plot(cmap=plt.cm.Blues, ax=ax)
plt.title("Matriz de Confusão (Random Forest Otimizado)")
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score

y_pred = rf5.predict(X_test)
roc_auc_score(y_test, y_pred)

In [ ]:
# Célula 12: Visualização da Curva ROC (Usando Scikit-learn e Matplotlib)

from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# 1. Obter as Probabilidades
# O ROC-AUC precisa da probabilidade de ser a classe positiva (Sobreviveu=1).
# O [:, 1] seleciona a coluna de probabilidade para a classe 1.
y_proba = rf5.predict_proba(X_test)[:, 1]

# 2. CALCULA A CURVA ROC
# 'fpr' (False Positive Rate) e 'tpr' (True Positive Rate) são os eixos do gráfico.
fpr, tpr, thresholds = roc_curve(y_test, y_proba)

# 3. CALCULA A ÁREA SOB A CURVA (AUC)
# Este é o valor numérico que você já calculou (0.7888...)
roc_auc = auc(fpr, tpr)

# 4. PLOTA O GRÁFICO
plt.figure(figsize=(8, 8))

# Plota a Curva ROC do modelo
plt.plot(fpr, tpr, color='darkorange', lw=2, 
         label=f'Curva ROC (área = {roc_auc:.4f})')

# Plota a linha de referência (classificador aleatório, 50% de chance)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Aleatório') 

plt.xlabel('Taxa de Falsos Positivos (FPR)')
plt.ylabel('Taxa de Verdadeiros Positivos (TPR)')
plt.title('Curva ROC para Random Forest Otimizado')
plt.legend(loc="lower right")

# Você pode salvar o arquivo aqui, se desejar.
plt.savefig("Curva_ROC_rf5.png", dpi=300, bbox_inches="tight")

plt.show()

In [ ]:
# Célula 13: Curva de Aprendizado (Usando Scikit-learn e Matplotlib)

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve, StratifiedKFold

# X e y devem ser os dados de treino completos (antes de X_train/X_test)
# Usando X e y completos, como o Yellowbrick faz, para avaliar o modelo RF5

# 1. Define o número de folds e o tamanho das amostras de treino
cv = StratifiedKFold(n_splits=12, shuffle=True, random_state=42)
train_sizes = np.linspace(0.3, 1.0, 10)

# 2. Calcula as pontuações da curva de aprendizado
train_sizes_abs, train_scores, test_scores = learning_curve(
    rf5,                 # O modelo otimizado
    X,                   # Conjunto de dados COMPLETO (X)
    y,                   # Rótulos COMPLETOS (y)
    cv=cv,
    train_sizes=train_sizes,
    scoring='f1_weighted',
    n_jobs=-1,           # Usa todos os cores
    random_state=42
)

# 3. Calcula as médias e desvios padrão para plotagem
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

# 4. PLOTA A CURVA DE APRENDIZADO
plt.figure(figsize=(8, 6))
plt.title("Curva de Aprendizado do Random Forest (F1 Score)")
plt.xlabel("Tamanho do Conjunto de Treino")
plt.ylabel("Score (F1 Ponderado)")

# Preenche a área de variação do treino
plt.fill_between(train_sizes_abs, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.1,
                 color="r")

# Preenche a área de variação da validação (teste)
plt.fill_between(train_sizes_abs, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.1, color="g")

# Plota as médias
plt.plot(train_sizes_abs, train_scores_mean, 'o-', color="r",
         label="Score de Treino")
plt.plot(train_sizes_abs, test_scores_mean, 'o-', color="g",
         label="Score de Validação Cruzada")

plt.legend(loc="best")
plt.grid()
plt.savefig("Curva_Aprendizado_rf5.png") # Salva o arquivo
plt.show()

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from yellowbrick.model_selection import LearningCurve
from  sklearn.model_selection import StratifiedKFold

fig, ax = plt.subplots(figsize=(6,4))
cv = StratifiedKFold(12)
sizes = np.linspace(0.3, 1.0, 10)
lc_viz = LearningCurve(
    rf5,
    cv = cv,
    train_sizes=sizes,
    scoring='f1_weighted',
    n_jobs=4,
    ax=ax,
)
plt.gcf().savefig("titanic/images/mlpr_0306.png", dpi=300, bbox_inches="tight")
lc_viz.fit(X, y)
lc_viz.poof()


In [ ]:
import janitor as jn
import pandas as pd

Xbad = pd.DataFrame(
    {
        "A": [1, None, 3],
        "  sales_numbers  ": [20.0, 30.0, None],
    }
)

In [ ]:
jn.clean_names(Xbad)

In [ ]:
def clean_col(name):
    return(
        name.strip().lower().replace(" ", "_")
    )

In [ ]:
Xbad.rename(columns=clean_col)

In [ ]:
Xbad.fillna(10)